In [1]:
import numpy as np
import starfile
import mrcfile
# import cv2
import pandas as pd

In [2]:
img_h = 600
bin_num = 20
fixed_ratio = True

In [3]:
def downsample(img, height):
    '''
    Downsample 2d array using fourier transform.
    factor is the downsample factor.
    '''
    m, n = img.shape[-2:]
    ds_factor = m / height
    width = int(n / ds_factor / 2) * 2
    F = np.fft.rfft2(img)
    A = F[..., 0:height // 2, 0:width // 2 + 1]
    B = F[..., -height // 2:, 0:width // 2 + 1]
    F = np.concatenate([A, B], axis=0)
    f = np.fft.irfft2(F, s=(height, width))
    return f

In [4]:
mrc = '../samples/input/01.mrc'
star = '../samples/input/01.star'
df = starfile.read(star)
img = mrcfile.read(mrc)

factor = img_h / img.shape[0]
img_w = int(img.shape[1] * factor)
img = downsample(img, img_h)
# img = cv2.resize(img, dsize=(img_w, img_h), interpolation=cv2.INTER_CUBIC)


FileNotFoundError: [Errno 2] No such file or directory: '../samples/input/01.mrc'

In [ ]:
print(df.head())
print(img.shape)

In [ ]:
df['rlnAutopickFigureOfMerit'] = np.random.uniform(0, 1, len(df))
print(df.head())

In [ ]:
a = df['rlnAutopickFigureOfMerit'].to_numpy()
if (a[0] == a).all():
    bin_num = 1

In [ ]:
out, bins = pd.cut(df["rlnAutopickFigureOfMerit"], bin_num, retbins=True)
print(bins)
dfs = tuple(df.groupby(out))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.imshow(img, binary_string=True)

i = 1
for df in dfs:
    df = df[1]
    fig.add_trace(go.Scatter(x = df['rlnCoordinateX'] * factor, 
                             y = df['rlnCoordinateY'] * factor, 
                             mode = 'markers',
                             marker = dict(
                                 symbol='circle-open',
                                 size = 5,
                                 color = 'red',
                                 opacity = i / bin_num,
                                 showscale=False,
                             ),
                             text = ['{:0.3f}'.format(i) for i in df['rlnAutopickFigureOfMerit']],
                             hovertemplate = '<i>Merit</i>: %{text}', 
                             name = "",
                             showlegend=False,
                            ))
    i += 1

merit_steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * i + [True] * (len(fig.data) - 1 - i)},
             ],
        label="{:0.3f}".format(bins[i]),
    )
    merit_steps.append(step)

marker_steps = []
for i in range(2,40,2):
    step = dict(
        method="restyle",
        args=[{"marker.size": i},
             ],
        label="{:0.3f}".format(i/factor),
    )
    marker_steps.append(step)

sliders = [dict(
            active=0,
            currentvalue={"prefix": "Threshold: "},
            steps=merit_steps,
            pad={"l": 10, "t": 0},
            ),
            dict(
            active=5,
            currentvalue={"prefix": "Marker size (px): "},
            steps=marker_steps,
            pad={"l": 10, "t": 100},
            ),
           ]


fig.update_layout(
    sliders=sliders,
    width=img_w,
    height=img_h,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',    
)

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.show(config={'responsive': False})
